# Exempel sur le topic modeling
Fatma Chamekh LO17 2021/2022


In [2]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Introduction

Les topic models (*modèles thématiques* en français) sont une famille de modèles qui sont capables de découvrir les topics (*thèmes* en français) d'une collection de documents textuels. Dans ce contexte, le terme "**topic**" désigne des groupes de mots qui se retrouvent souvent ensemble dans un même document. Par exemple, dans un recueil d'articles de journaux, un topic model peut identifier un topic composé des mots :
- "homme politique", "droit" et "parlement", et un autre caractérisé par 
- "joueur", "match" et "carton rouge"

Les topics modeling ne peuvent pas affecter un titre à ces topics : c'est notre tâche d'interpréter ces topics et de leur donner des étiquettes telles que **politique** et **football**.

La particularité de ces méthodes c'est que nous n'avons pas de labels : ce sont des tâches **non-supervisées**.

L'un des modèles les plus populaires est le LDA. Le LDA est un modèle génératif qui considère chaque document comme un mélange de topics. Ce sont ces topics qui seront en charge de générer les mots. Par exemple, 

- le topic **football** générera le mot "ballon" avec une probabilité élevée, 
- tandis que le topic **politique** aura une probabilité beaucoup plus élevée pour générer le mot "politicien" que pour générer le mot "ballon"

![image.png](attachment:image.png)

## Données : Grand Débat National

- L'un des contextes où la modélisation des topics est très utile est celui des questions ouvertes. Il nous permet d'explorer les différents topics abordés dans les réponses des gens. 

- Dans ce notebook nous allons explorer un ensemble complet de réponses du [Grand Débat National](https://granddebat.fr/), un débat public organisé par le président Macron. Le but du débat était de mieux comprendre les besoins et les opinions des Français suite aux manifestations des gilets jaunes. Les résultats de ce débat sont maintenant disponibles sous forme de [données ouvertes](https://granddebat.fr/pages/donnees-ouvertes).
- Il faut choisir 

## 2. Charger les données

Nous allons tout d'abord importer un des fichiers csv sur la transition écologique dans un dataframe [pandas](https://pandas.pydata.org/).

In [3]:
%%capture captured

import pandas as pd

chemin = "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg.cloud.ovh.net/2019-02-06/LA_TRANSITION_ECOLOGIQUE.csv" # TODO

raw_data = pd.read_csv("data/LA_TRANSITION_ECOLOGIQUE.csv", error_bad_lines=False, warn_bad_lines=False)

In [4]:
raw_data

,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,authorZipCode,...,"Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?",Quelles seraient pour vous les solutions les plus simples et les plus supportables sur un plan financier pour vous inciter à changer vos comportements ?,"Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?","Si oui, que faudrait-il faire pour vous convaincre ou vous aider à changer de mode de chauffage ?","Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?","Si oui, que faudrait-il faire pour vous convaincre ou vous aider à utiliser ces solutions alternatives ?","Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?",Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?,Que pourrait faire la France pour faire partager ses choix en matière d'environnement au niveau européen et international ?,Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?
0,2-4,transition écologique,2019-01-22 09:37:49,2019-01-22 09:37:49,NaN,False,NaN,VXNlcjoxMTQwMTc0YS0xZTFmLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,97231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Enseignement du tri sélectif à l'école,Multiplier les centrales géothermiques
1,2-5,La surpopulation,2019-01-22 09:39:33,2019-01-22 09:39:33,NaN,False,NaN,VXNlcjpjOWYxZWQ1NS0xYzEwLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,57000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mettre en oeuvre au niveau national ses engage...,Les problèmes auxquels se trouve confronté l’e...
2,2-6,climat,2019-01-22 09:39:50,2019-01-22 09:39:50,NaN,False,NaN,VXNlcjozZjlhNzAwOS0xYTc2LTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,34140,...,"développer les transports en commun ,",NaN,Non,NaN,Oui,NaN,"Les transports en commun, L'auto partage, Le t...",NaN,NaN,NaN
3,2-7,POLLUTION AIR EAU,2019-01-22 09:40:19,2019-01-22 09:40:19,NaN,False,NaN,VXNlcjozOWQwNzJjNC0xZDEwLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,17400,...,NaN,Plus de transports publics dans les petites co...,Non,NaN,Non,Le co-voiturage ne correspond pas toujours aux...,Les transports en commun,L'Etat,Elle n'a aucun pouvoir. Impossible de contrain...,NaN
4,2-8,Economie vs Ecologie,2019-01-22 09:41:33,2019-01-22 09:41:33,NaN,False,NaN,VXNlcjo3M2YxN2NlZS0xZDRiLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,35430,...,"Aménagement de piste cyclable, développement d...","Détaxer le mode de chauffage écologique, une a...",Oui,une aide significative pour de l'éolien ou du ...,Non,Une piste cyclable pour éviter de risquer ma v...,"Les transports en commun, Le covoiturage, Le vélo","La commune, le département, la région",Demander à Nicolas Hulot,Une vrai politique écologique et non économique
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35977,2-41683,Réduire l'électricité,2019-01-31 18:59:16,2019-01-31 18:59:16,NaN,False,NaN,VXNlcjo3NzIwZTZhYy0yNGJiLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,78130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"concernant les mails, faire en sorte qu'ils so...",NaN
35978,2-41686,Contraindre les entreprises à produire selon d...,2019-01-31 18:59:52,2019-01-31 18:59:52,NaN,False,NaN,VXNlcjpiZTI3NWViOC0yNTc4LTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,75017,...,Pourquoi ne pas mettre toutes les grandes vill...,Ce n'est pas au consommateur de supporter le p...,Oui,Démocratiser la géothermie et la rendre access...,Oui,C'est vraiment une vision liée aux hydrocarbur...,"Le transport à la demande, L'auto p

Chacune des lignes de ce dataframe ``raw_data`` contient des réponses répondant à une liste de questions sur la transition écologique. Certaines de ces questions sont à choix multiples, tandis que d'autres sont des questions ouvertes. 

In [5]:
raw_data.columns

Index(['reference', 'title', 'createdAt', 'publishedAt', 'updatedAt',
       'trashed', 'trashedStatus', 'authorId', 'authorType', 'authorZipCode',
       'Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?',
       'Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?',
       'Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?',
       'Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?',
       'À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?',
       'Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?',
       'Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?',
   

Nous allons étudier la dernière question : 

``Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?`` 

car cette dernière laisse plus de liberté aux personnes. Notre objectif est d'analyser les topics sur lesquels portent leurs réponses grâce au modèle LDA.

In [6]:
question = "Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?"
data = raw_data[question]

In [8]:
data.to_csv('data/test_example.csv',index=False)

In [7]:
print(data)

0                   Multiplier les centrales géothermiques
1        Les problèmes auxquels se trouve confronté l’e...
2                                                      NaN
3                                                      NaN
4          Une vrai politique écologique et non économique
                               ...                        
35977                                                  NaN
35978    Il faut que la France montre l'exemple et de m...
35979    Toutes les collectivités et administrations do...
35980    je souhaite insister sur l'intérêt de l'énergi...
35981    Nous allons vers une crise énergétique d'ici à...
Name: Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?, Length: 35982, dtype: object


In [6]:
data.head()

0               Multiplier les centrales géothermiques
1    Les problèmes auxquels se trouve confronté l’e...
2                                                  NaN
3                                                  NaN
4      Une vrai politique écologique et non économique
Name: Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?, dtype: object

Nous pouvons remarquer qu'il y a beaucoup de données manquantes (comme toute question ouverte, les personnes décident oui ou non d'écrire un commentaire). Une étape de néttoyage est donc nécessaire.

## 3. Nettoyer et vectoriser les documents

Avant d'entraîner notre modèle LDA, nous avons besoin de tokenizer notre texte. Nous allons tokenizer grâce à la librarie [spaCy](https://spacy.io/) car nous allons effectuer seulement quelques prétraitements de base. Nous allons juste initialiser un modèle vierge pour la langue française.

**Remarques :** Après avoir installé la librairie SpaCy (pip install spacy), il faut télécharger un pipeline entraîné pour la langue française.

``python -m spacy download fr_core_news_sm``

spaCy fournit une variété d'annotations linguistiques pour vous donner un aperçu de la structure grammaticale d'un texte. Cela inclut entre autres des preprocessing classique en NLP à savoir la **lemmatisation**.

In [7]:
import spacy

nlp = spacy.load("fr_core_news_sm")

Supprimons toutes les lignes du dataframe qui n'ont pas de réponse pour notre question (les `NaN`s ci-dessus). Ce nouveau dataframe s'appellera ``textes``

In [19]:
print(type(data))
print(len(data))
def delete_NaN_lines(input_data):
    remove_nan = input_data.dropna()
    return remove_nan
textes = delete_NaN_lines(data)

<class 'pandas.core.series.Series'>
35982


In [20]:
textes.head()

0               Multiplier les centrales géothermiques
1    Les problèmes auxquels se trouve confronté l’e...
4      Une vrai politique écologique et non économique
5    Les bonnes idées ne grandissent que par le par...
6    Pédagogie dans ce sens là dés la petite école ...
Name: Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?, dtype: object

Ensuite, nous utilisons spaCy pour effectuer notre premier prétraitement (cela peut prendre quelques minutes) :

In [21]:
%%time

spacy_docs = list(nlp.pipe(textes))

CPU times: total: 5min 6s
Wall time: 5min 18s


In [24]:
print(type(spacy_docs))
print(len(spacy_docs))

<class 'list'>
25066


Nous avons maintenant une liste de documents spaCy. Nous allons transformer chaque document spaCy en une liste de tokens. Au lieu des tokens originaux, nous allons travailler avec les lemmes à la place. Cela permettra à notre modèle de mieux généraliser. En effet, nous voulons par exemple que "continuation" et "continuations" représentent la même signification. Voici la liste complète des prétraitements : 
 
1. supprimer tous les **mots de moins de 3 caractères**,
2. supprimer tous les **stop-words**, et
3. **lemmatiser** les mots restants et,
4. mettre ces mots en **minuscules**.

In [25]:
%%time

docs = []
for doc in spacy_docs:
    tokens = []
    for token in doc:
        if len(token.orth_) > 3 and not token.is_stop: # prétraitements 1 et 2
            token = token.lemma_.lower()
            tokens.append(token)
    docs.append( tokens )

CPU times: total: 2.09 s
Wall time: 2.1 s


Voici un aperçu du **premier** document tokenisé :

In [26]:
print(docs[0])

['multiplier', 'centrale', 'géothermique']


**deuxième** document tokenisé :

In [27]:
print(docs[1])

['problème', 'trouve', 'confronter', 'ensemble', 'planète', 'dénoncent', 'parfaire', 'désordre', 'gilet', 'jaune', 'france', 'il', 'surpopulation', 'mondial', 'population', 'passer', 'd’1,5', 'milliard', 'habitant', '1900', 'milliard', '2020', 'monter', 'bientôt', 'milliard', '2040', 'progrès', 'communication', 'village', 'mondial', 'individu', 'fondre', 'asie', 'fondre', 'afrique', 'passer', 'quartiers', 'campagne', 'pays', 'aspir', 'vivr', 'blâmer', 'lotir', 'concitoyen', 'logement', 'nourriture', 'bien', 'consommation', 'déplacement', 'etc.', 'mère', 'problème', 'bien', 'solution', 'problème', 'stabilisation', 'croissance', 'démographique', 'partage', 'richesse', 'partage', 'terre', 'partage', 'protection', 'biodiversité', 'règlement', 'conflit', 'lutte', 'contre', 'déforestation', 'lutte', 'contre', 'dérèglement', 'climatique', 'règlement', 'conflit', 'stabilisation', 'migration', 'concurrence', 'commercial', 'mondial', 'etc.', 'français', 'européen', 'mondial', 'france', 'jouer', 

Afin de conserver un peu l'ordre des mots lors de notre modélisation, nous allons tenir en compte les **bigrammes fréquents**. Pour cela, nous allons utiliser la bibliothèque [Gensim](https://radimrehurek.com/gensim/) (excellente bibliothèque NLP pour les topics modeling et word embeddings)

Voici le processus retenu : 

- **Identifier les bigrammes fréquents** dans le corpus, 
- **Ajouter à la liste des tokens** pour les documents dans lesquels ils apparaissent. 

Cela signifie que les bigrammes ne seront pas dans leur position correcte dans le texte, mais comme les topic models sont des modèles de **bag-of-words** (*sac-de-mots* en français) qui ignorent la position des mots, cela ne pose pas de problèmes.

In [35]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)
i = 0
for index in range(len(docs)):
    for token in bigram[docs[index]]:
        if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
            docs[index].append(token)

In [36]:
print(docs[4])

['pédagogie', 'sens', 'petit', 'école', 'sensibilisation', 'parc', 'naturel', 'enfant', 'devenir', 'prescripteur', 'génération', 'futur', 'urgence', 'génération_futur']


Passons à la dernière étape du prétraitement spécifique à Gensim. Nous allons tout d'abord créer une représentation dictionnaire des documents. Ce dictionnaire mappera chaque mot à un identifiant unique et nous aidera à créer des représentations en sac-de-mot de chaque document. Ces représentations en sac-de-mots contiennent les identificateurs des mots du document ainsi que leur fréquence (ici en nombre d'occurences). De plus, nous pouvons supprimer les mots les moins fréquents et les plus fréquents du vocabulaire. Cela améliorera la qualité de notre modèle et accélèrera son entraînement.

In [37]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Nombre de mots unique dans les documents initiaux :', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.25)
print('Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents :', len(dictionary))

print("Exemple :", dictionary.doc2bow(docs[4]))

Nombre de mots unique dans les documents initiaux : 33299
Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents : 12849
Exemple : [(170, 1), (171, 1), (172, 1), (173, 1), (174, 1), (175, 1), (176, 1), (177, 1), (178, 1), (179, 1), (180, 1), (181, 1), (182, 1)]


Ensuite, nous créons des représentations en Bag-of-Words pour chaque document du corpus voir la méthode [doc2bow](https://radimrehurek.com/gensim/corpora/dictionary.html) :

In [43]:
corpus = [dictionary.doc2bow(docs[i]) for i in range(len(docs))]
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 2),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 2),
  (37, 1),
  (38, 2),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 2),
  (54, 2),
  (55, 1),
  (56, 3),
  (57, 4),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 3),
  (65, 1),
  (66, 2),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 3),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 2),
  (77, 1),
  (78, 1),
  (79, 2),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1)],
 [(86, 1), (87, 1), (88, 1), (89, 1)],
 [(22, 1),
  (64, 1),
  (70

## 4. Topic Modeling avec LDA

Maintenant, il est temps d'entraîner notre LDA ! Pour ce faire, nous utilisons les paramètres suivants : 

- **corpus** : les représentations en sac-de-mots de nos documents
- **id2token** : le mappage des index aux mots
- **num_topics** : le nombre de topics que le modèle doit identifier (fixons à <font color = "red"><b>10</b></font>)
- **chunksize** : le nombre de documents que le modèle voit à chaque mise à jour (fixons à <font color = "red"><b>1 000</b></font>)
- **passes** : le nombre de fois où nous montrons le corpus total au modèle pendant l'entraînement (fixons à <font color = "red"><b>5</b></font>)
- **random_state** : nous utilisons une graine pour assurer la reproductibilité (fixons à <font color = "red"><b>1</b></font>)

Remarquons que l'entraînement peut durer quelques minutes

In [48]:
%%time
from gensim.models import LdaModel

model = LdaModel(corpus,id2word = dictionary,num_topics=10,chunksize=1000,passes=5,random_state=1)

CPU times: total: 1min 48s
Wall time: 1min 3s


In [49]:
for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.019*"zone" + 0.018*"ville" + 0.018*"centre" + 0.017*"protection" + 0.015*"commercial" + 0.013*"école" + 0.013*"environnement" + 0.012*"commerce" + 0.012*"espace" + 0.012*"espèce"

***********
* topic 2 *
***********
2 : 0.039*"énergie" + 0.029*"nucléaire" + 0.027*"électrique" + 0.025*"voiture" + 0.024*"véhicule" + 0.012*"renouvelable" + 0.012*"éolien" + 0.011*"solution" + 0.011*"production" + 0.011*"électricité"

***********
* topic 3 *
***********
3 : 0.017*"aide" + 0.017*"panneau" + 0.016*"chauffage" + 0.013*"logement" + 0.012*"bâtiment" + 0.012*"solaire" + 0.010*"construction" + 0.010*"public" + 0.010*"euro" + 0.009*"énergie"

***********
* topic 4 *
***********
4 : 0.025*"faire" + 0.023*"falloir" + 0.011*"bien" + 0.010*"prendre" + 0.009*"changer" + 0.009*"pouvoir" + 0.008*"planète" + 0.008*"monde" + 0.007*"problème" + 0.007*"france"

***********
* topic 5 *
***********
5 : 0.025*"transport" + 0.011*"ville" + 0.008*"grand" + 0.008*"local" + 

## 5. Résultats et visualisation

Voyons ce que le modèle a appris. Pour ce faire, affichons les dix mots les plus caractéristiques pour chacun des topics. Nous obeservons déjà des tendances intéressantes : si certains topics sont plus généraux (comme le topic 3), d'autres font référence à des topics très pertinents : 
- véhicules électriques (**topic 1**), 
- énergie (alternative) (**topic 2**), 
- agriculture (**topic 6**), 
- déchets et recyclage (**topic 7**) et 
- fiscalité (**topic 9**).

In [ ]:
for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.041*"produit" + 0.035*"agriculture" + 0.023*"pesticide" + 0.022*"environnement" + 0.018*"interdir" + 0.017*"agriculteur" + 0.016*"animal" + 0.015*"santé" + 0.015*"industriel" + 0.014*"chasse"

***********
* topic 2 *
***********
2 : 0.043*"terme" + 0.027*"long" + 0.022*"logement" + 0.017*"bâtiment" + 0.017*"mode" + 0.015*"construction" + 0.015*"court" + 0.014*"long_terme" + 0.011*"chauffage" + 0.011*"public"

***********
* topic 3 *
***********
3 : 0.021*"falloir" + 0.017*"faire" + 0.012*"bien" + 0.012*"france" + 0.010*"pays" + 0.010*"planète" + 0.009*"problème" + 0.008*"être" + 0.008*"politique" + 0.008*"monde"

***********
* topic 4 *
***********
4 : 0.070*"énergie" + 0.042*"nucléaire" + 0.019*"renouvelable" + 0.017*"éolien" + 0.017*"développer" + 0.014*"centrale" + 0.013*"production" + 0.012*"fossile" + 0.012*"recherche" + 0.012*"falloir"

***********
* topic 5 *
***********
5 : 0.010*"public" + 0.010*"investissement" + 0.009*"pouvoir" + 0.0

Une autre façon d'observer les topics est de les **visualiser**. Ceci peut être fait avec la bibliothèque [pyLDAvis](https://github.com/bmabey/pyLDAvis). PyLDAvis nous illustre :

- à quel point les sujets sont **populaires** dans notre corpus, 
- à quel point les sujets sont **similaires**
- et quels sont les **mots les plus importants** pour ce sujet. 

Notez que cela peut prendre quelques minutes pour charger.

In [51]:
import pyLDAvis.gensim_models
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)

C:\Users\Ho Xuan Vinh\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.187977 -0.233988       1        1   4.500172
1      0.032788  0.134224       2        1  14.324397
2     -0.054717  0.140872       3        1   6.745194
3      0.030016  0.144662       4        1  21.611057
4      0.092907  0.048378       5        1  13.528324
5     -0.254224 -0.236127       6        1   2.776269
6      0.160002 -0.139488       7        1   8.551725
7      0.084029 -0.029398       8        1  10.764438
8      0.015287  0.172811       9        1  14.239655
9     -0.294066 -0.001947      10        1   2.958769, topic_info=                 Term         Freq        Total Category  logprob  loglift
466        transition  4722.000000  4722.000000  Default  30.0000  30.0000
88         écologique  6119.000000  6119.000000  Default  29.0000  29.0000
256           énergie  5251.000000  5251.000000  Default  28.0000  28.0000
544           produit  4535.000000  4535.000000  Default  27.0000  27.0000
620         nucléaire  3237.000000  3237.000000  Default  26.0000  26.0000
...               ...          ...          ...      ...      ...      ...
361          économie   140.316608  1604.697296  Topic10  -5.0848   1.0836
2065           accord   116.726095   334.302376  Topic10  -5.2689   2.4682
1403             être   130.803585  1841.572688  Topic10  -5.1550   0.8757
657        subvention   117.573605   425.483073  Topic10  -5.2616   2.2342
470   énergie_fossile   118.044172   639.910775  Topic10  -5.2576   1.8301

[659 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3781      7  0.992093   \n    
8883      7  0.995493  \n     
3544      2  0.093601     2018
3544      3  0.085801     2018
3544     10  0.819012     2018
...     ...       ...      ...
1403      9  0.080366     être
1403     10  0.071135     être
641       5  0.016472    œuvre
641       8  0.043924    œuvre
641       9  0.933393    œuvre

[1416 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Enfin, examinons les topics que le modèle reconnaît dans certains des documents individuels. Nous voyons ici comment LDA tend à attribuer une probabilité élevée à un faible nombre de sujets pour chaque document, ce qui rend ses résultats très interprétables.

In [ ]:
# Nous en affichons que 4
n_doc = 4
i = 0
for (text, doc) in zip(textes[:n_doc], docs[:n_doc]):
    i += 1
    print("***********")
    print("* doc", i, "  *")
    print("***********")
    print(text)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.1])
    print()

***********
* doc 1   *
***********
Multiplier les centrales géothermiques
[(4, 0.77492505)]

***********
* doc 2   *
***********
Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France ne sont-ils pas dus, avant tout, à la surpopulation mondiale ? Cette population est passée d’1,5 milliards d’habitants en 1900 à 7 milliards en 2020 et montera bientôt à 10 milliards vers 2040.  Avec les progrès de la communication dans ce village mondial, chaque individu, du fin fond de l’Asie au fin fond de l’Afrique, en passant par les « quartiers » et les « campagnes » de notre pays, aspire à vivre – et on ne peu l’en blâmer – comme les moins mal lotis de nos concitoyens (logement, nourriture, biens de consommation, déplacement,etc.).  Voilà la mère de tous les problèmes. Si tel est bien le cas, la solution à tous les problèmes (stabilisation de la croissance démographique, partage des richesses, partage des ter

**Conclusion**

De nombreuses collections de textes non structurés ne sont pas accompagnées de labels. Les topic models (*modèles thématiques* en français) tels que le LDA sont une technique utile pour découvrir les topics les plus importants dans ces documents. **Gensim** facilite l'apprentissage sur ces sujets et **pyLDAvis** présente les résultats d'une manière visuellement attrayante. Ensemble, ils forment une puissante boîte à outils pour mieux comprendre et explorer des sous-ensembles de textes connexes. Si ces résultats sont souvent déjà très révélateurs, il est également possible de les utiliser comme point de départ, par exemple pour un exercice de labellisation pour la classification supervisée de textes. En somme, les modèles thématiques devraient figurer dans la boîte à outils de chaque data scientist comme un moyen très rapide d'obtenir un aperçu des grandes collections de documents.